In [1]:
words =  open("names.txt", "r").read().splitlines()
words[0]

'emma'

In [2]:
import torch
import torch.nn.functional as F


In [127]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

X, Y = [], []
context_size = 3

for word in words:
    context = [0] * context_size
    for ch in word + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [128]:
print(X.shape)
print(Y.shape)

torch.Size([228146, 3])
torch.Size([228146])


In [464]:
h_l = 200
emb_space = 20

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, emb_space), generator=g, requires_grad=True)

W1 = torch.randn((X.shape[1]*C.shape[1], h_l), generator=g, requires_grad=True)
b1 = torch.randn((h_l), generator=g, requires_grad=True)

W2 = torch.randn((h_l, 27), generator=g, requires_grad=True)
b2 = torch.randn((27), generator=g, requires_grad=True)

parameters = [C, W1, b1, W2, b2]

W1.shape

torch.Size([60, 200])

In [462]:
print(C.shape)
print(X.shape)

torch.Size([27, 20])
torch.Size([228146, 3])


In [460]:
#forward pass
for _ in range(10000):
    # mini_batch
    batch = torch.randint(0, X.shape[0], (32,)) 
    emb = (C[X[batch]]).view(-1, X.shape[1]*C.shape[1])
    h = torch.tanh(emb@W1 + b1)
    logits = h@W2 + b2
    loss = F.cross_entropy(logits, Y[batch])
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.0001 * p.grad
print(loss.item())

2.695152759552002


In [463]:
emb.shape

torch.Size([228146, 60])

In [461]:
emb = (C[X]).view(-1, X.shape[1]*C.shape[1])
h = torch.tanh(emb@W1 + b1)
logits = h@W2 + b2
loss = F.cross_entropy(logits, Y)
print(loss.item())

2.1922225952148438


In [423]:
X.shape[0]

228146